# Automating Calcium assay data analysis

Annotations specific to this run through for assessment are surrounded by ###:

###They are not included in the usual code.###

## Importing required packages

In [ ]:
# These are the packages we'll be using:

import pandas as pd
import matplotlib.pyplot as plt
import math

## Reading in data
I found that I could only import data from my github if i called it in the raw format, here i've done this automatically so that you can copy the URL directly from the github website (or the test data from my README).

In [ ]:
url = '' #<--- insert URL here

print("The URL is:", url)


raw_url = url.replace("blob", "raw") # Replacing blob with raw, data doesn't seem to import unless it is in 'raw' format

print("\nOur converted raw URL is:", raw_url)


test_dataset = pd.read_csv(raw_url)# Reading in our data

print("\nThis is the whole dataset below:\n\n", test_dataset)

###There are many unwanted columns here### 

## Tidying up test_dataset

The columns of interest are Time (sec) and the R[i] R1 columns.
The first R refers to the Region, in this case individual cells.
The second R1 is the ratio between W1 and W2, this is what we're interested in and as the ratio has already been calculated for us we no longer need W1 and W2.

### Deleting unwanted columns

In [ ]:
# Identifying and deleting R[i] W1 Avg columns
Ri_W1_Avg_columns = [] # Creating a list to add the columns too 

for i in range(1, 51): # Looping through 50 possible columns # Looping through 50 possible columns
    column_name = 'R' + str(i) + ' W1 Avg' # Identifying the columns of interest
    if column_name in test_dataset.columns: # IF function to ensure the column identified is in the corect dataset
        Ri_W1_Avg_columns.append(column_name) # IF it is, adding the coluymn to a list

test_dataset = test_dataset.drop(columns=Ri_W1_Avg_columns) # Deleting the columns identifed in the list

    
# Identifying and deleting R[i] W2 Avg columns
Ri_W2_Avg_columns = [] # Creating a list to add the columns too 

for i in range(1, 51): # Looping through 50 possible columns # Looping through 50 possible columns
    column_name = 'R' + str(i) + ' W2 Avg' # Identifying the columns of interest
    if column_name in test_dataset.columns: # IF function to ensure the column identified is in the corect dataset
        Ri_W2_Avg_columns.append(column_name) # IF it is, adding the coluymn to a list

test_dataset = test_dataset.drop(columns=Ri_W2_Avg_columns) # Deleting the columns identifed in the list

    
# Deleting columns where all the values are NaN
nan_columns = test_dataset.columns[test_dataset.isnull().all()].tolist() # Identifying the columns

test_dataset = test_dataset.drop(columns=nan_columns) # Dropping them from the DF


# Deleting columns that contain only 0 and NaN values

#Identifying the columns that contain only 0 or NaN values
na_null_columns = test_dataset.columns[test_dataset.isin([0, pd.np.nan]).all() & test_dataset.notnull().any(axis=0)]
        

test_dataset = test_dataset.drop(columns=na_null_columns) # Dropping them from the DF


# Deleting unnamed columns
unnamed_columns = [col for col in test_dataset.columns if "Unnamed:" in col] # Identifying the unnamed columns

test_dataset = test_dataset.drop(columns=unnamed_columns) # Dropping them from the DF


print(" This is the test dataset without: R[i] W1 or W2 Avg columns, NaN columns, blank columns or unnamed columns:\n\n", test_dataset)

### Naming our data

In [ ]:
time = test_dataset["Time (sec)"] # Assigning the 'Time (sec)' column the name 'time'


# Assigning the R[i] R1 columns the name calcium_r[i]
for i in range(1, 51): # Looping through 50 possible columns # Looping through 50 possible columns
    try: # Using try and except in the loop to avoid errors relating to missing columns in datasets of less than 50 cells
        exec(f'calcium_r{i} = test_dataset["R{i} R1"].values') # Assigning names
    except KeyError:

        continue


print("Our time dataset looks like this:\n\n", time, "\n\n\n\n And an example of our calcium_r[i] (9) dataset looks like this:\n\n", calcium_r9)

## Visualising all of our calcium_r[i] data against time

In [ ]:
#Setting the number of rows based on the number of columns that are in the original df
num_row = None # Creating a variable

for i in range(1, 51): # Looping through 50 possible columns
    var_name = "calcium_r" + str(i) # Creating a variable
    if var_name in locals(): # IF function to see if the current iteration of calcium_r[i] is defined
        num_row = i # Assigning the highest value to our variable

num_row_half = math.ceil(num_row / 2) # Divides the number of rows by 2 (as there are 2 columns) and rounding up


# Plotting the data
nrows = math.ceil(num_row_half) # Setting no. of rows (#+1 as sometimes column calcium_r1 is not the first column with data)
ncols = 2 # Setting no. of columns

fig, axs = plt.subplots(nrows, ncols, figsize=(10, nrows*5)) # Creating figure/subplots

axs = axs.flatten() # Flattening the subplot array to make it easier to iterate over

# Looping through the variables to plot each one
for i, ax in enumerate(axs): # Looping through 50 possible columns
    calcium_r = "calcium_r" + str(i+1) # Calling calcium_r[i] for the current iteration
    if calcium_r in locals():
        ax.plot(time, locals()[calcium_r], 'b.') # Plotting time on the x axis
        ax.set_title("Calcium {}/Time".format(i+1)) # Setting the title
        ax.set_ylabel("Calcium {}".format(i+1)) # Setting the y axis label
        ax.set_xlabel("Time (sec)") # Setting the x axis label

## Standardising calcium_r[i] data

To be able to compare between cells with varying peaks and baselines I am standardising the output of y on a scale of 0 and 1 by dividing all values by the peak value

In [ ]:
# Defining the peak of each dataset.
for i in range(1, 51):
    try: # Using try and except in the loop to avoid errors relating to missing columns in datasets of less than 50 cells
        dataset = eval(f"calcium_r{i}") # Defining the dataset
        peak = max(dataset) # Setting the peak to the maximum value in the dataset
        exec(f"peak{i} = {peak}") # Saving the value as a new peak[i] variable
    except NameError:
        pass

    
# Dividing each value by the dataset's corresponding peak
for i in range(1, 51):
    try: # Using try and except in the loop to avoid errors relating to missing columns in datasets of less than 50 cells
        peak = eval(f"peak{i}")  # Set peak value for the current iteration
        calcium_r = eval(f"calcium_r{i}")  # Get the calcium_r[i] for the current iteration
        standardised_calcium_r = [value / peak for value in calcium_r] # Standardise each point of data by dividing by peak
        exec(f"standardised_calcium_r{i} = standardised_calcium_r") 
                # Saving the results as a new variable
    except NameError:
        pass

### Visualising standardised data

In [ ]:
# Plotting the data
nrows = math.ceil(num_row_half)  # Setting no. of rows (#+1 as sometimes column calcium_r1 is not the first column with data)
ncols = 2  # Setting no. of columns

fig, axs = plt.subplots(nrows, ncols, figsize=(10, nrows * 5))  # Creating figure/subplots

axs = axs.flatten()  # Flattening the subplot array to make it easier to iterate over

# Looping through the variables to plot each one
for i, ax in enumerate(axs):  # Looping through 50 possible columns
    standardised_calcium_r = f"standardised_calcium_r{i + 1}"  # Calling standardised_calcium_r[i] for the current iteration
    if standardised_calcium_r in locals():
        ax.plot(time, locals()[standardised_calcium_r], "b.")  # Plotting time on the x axis
        ax.set_title("Standardised Calcium {}/Time".format(i+1)) # Setting the title
        ax.set_ylabel("Standardised Calcium {}".format(i+1)) # Setting the y axis label
        ax.set_xlabel("Time (sec)") # Setting the x axis label

The data seems to consist of swift incline (the action of a drug) which comes to a peak followed by a small lag period and a swift decline. This decline then plateaus to a trough, and finally the data tails off and rises again.
I am interested in identifying the steep rate of decline after the lag phase but before the decline begins to plateau - this should be a fairly consistent measure of how quickly calcium levels return to normal.

## Identifying the rate at which calcium levels return to normal following the peak

This code was designed to call a range of datasets with up to 50 regions/columns.

In [ ]:
# Combining standardised_calcium_r[i] and time into a list of tuples
for i in range(1, 51): # Looping through 50 possible columns
    try: # Using try and except in the loop to avoid errors relating to missing columns in datasets of less than 50 cells
        globals()['time_tuples' + str(i)] = list(zip(time, locals()['standardised_calcium_r' + str(i)])) # Combines the matching indexes
                # of time and standardised_calcium_r[i] into tuples
    except KeyError:
        pass


# Calculating the gradient between the tuples
for i in range(1, 51): # Looping through 50 possible columns
    try: # Using try and except in the loop to avoid errors relating to missing columns in datasets of less than 50 cells
        gradients = [] # Clearing the gradients list for this iteration 
        for j in range(len(globals()['time_tuples' + str(i)])-1): # Looping through the 50 possible time_tuples
            gradient = (globals()['time_tuples' + str(i)][j+1][1] - globals()['time_tuples' + str(i)][j][1]) / (globals()['time_tuples' + str(i)][j+1][0] - globals()['time_tuples' + str(i)][j][0])
                    # Defining each gradient
            gradients.append(gradient) # Appending each gradient to a list of gradients
        globals()['gradients' + str(i)] = gradients # Naming the gradients list with the number of the corresponding
                # standardised_calcium_r[i]
    except KeyError:
        pass


# Defining a function to find the longest straight(ish) declining line (this should be the line of interest) based on a
    # tolerance of 'straightness'.
def calculate_average_gradient(times, values, tolerance): # Defining the fucniton and it's arguments
  time_tuples = list(zip(times, values)) # 
  gradients = [] # Clearing the gradients list for this iteration 
  for i in range(len(time_tuples)-1): # Looping through 50 possible time_tuples
    gradient = (time_tuples[i+1][1] - time_tuples[i][1]) / (time_tuples[i+1][0] - time_tuples[i][0]) # Defining the
            # gradient between each of the points 
    gradients.append(gradient) # Adding this gradient to the list of graients
  max_length = 0 # Setting our variables to 0
  start_index = 0 # Setting our variables to 0
  end_index = 0 # Setting our variables to 0
  previous_gradient = gradients[0] # Moving on to the next gradient
  for i in range(len(time_tuples)-1):
    length = 0 # Setting our variables to 0
    for j in range(i, len(time_tuples)-1): # Looping through 50 possible time_tuples
      average_gradient = (time_tuples[j+1][1] - time_tuples[i][1]) / (time_tuples[j+1][0] - time_tuples[i][0])
              # Calculating the average gradient of the line
      percentage_change = (gradients[j] - average_gradient) / average_gradient # Determining the percentage change between
              # the average gradient before and after adding this gradient
      if abs(percentage_change) < tolerance and average_gradient < 0: # Checking that the percentage change is within the
              # set tolerance, and that the line has an overall negative gradient
        length += 1 # Adds +1 to the length of the line if these conditions are met.
        previous_gradient = average_gradient # Moving on to the next gradient
      else: # If the above statement fails, this line breaks
        break
    if length > max_length: # If the line is the largest found so far it is saved
      max_length = length
      start_index = i
      end_index = i+length
  times1, values1 = zip(*time_tuples[start_index:end_index+1]) # Fetches the start and end index from the tuples
  average_gradient = (values1[-1] - values1[0]) / (times1[-1] - times1[0]) # Calculates the average gradient of the line
  return average_gradient, start_index, end_index # Returns the average gradient adn the start and end index of the max line


# Setting the tolerances for all standardised_calcium_r[i] to a default 0.3 - seems to work in the majority of cases
for i in range(1, 51): # Looping through 50 possible columns
    if f'standardised_calcium_r{i}' in globals(): # Only sets a tolerance if the standardised_calcium_r[i] is defined
        exec(f'tolerance{i} = 0.3') # Sets the tolerance[i] to 0.3

In [ ]:
# Running the function on all standardised_calcium_r[i] to find the line of interest in each 
for i in range(1, 51): # Looping through 50 possible columns
  try: # Using try and except in the loop to avoid errors relating to missing columns in datasets of less than 50 cells
    standardised_calcium_r_values = globals()["standardised_calcium_r"+str(i)]  # Get the standardised_calcium_r values for the current iteration
    tolerance = globals()["tolerance"+str(i)]  # Get the tolerance value for the current iteration
    average_gradient, start_index, end_index = calculate_average_gradient(time, standardised_calcium_r_values, tolerance) #Runs the
            # function wiht the arguments unique to each standardised_calcium_r[i]
    globals()["average_gradient"+str(i)] = average_gradient
    globals()["start_index"+str(i)] = start_index
    globals()["end_index"+str(i)] = end_index
  except KeyError:
    continue

# Extracting the start and end index for the lines - this will be useful in plotting them later! This was done in the
        # function but were not saved as [i], so were overwritten for each iteration.
for i in range(1, 51): # Looping through 50 possible columns
  try:# Using try and except in the loop to avoid errors relating to missing columns in datasets of less than 50 cells
    start_index = globals()["start_index"+str(i)]  # Sets the variable to the one defined in the current iteration [i]
    end_index = globals()["end_index"+str(i)]  # Sets the variable to the one defined in the current iteration [i]
    time_tuples_values = globals()["time_tuples"+str(i)]  # Sets the variable to the one defined in the current iteration [i]
    times, values = zip(*time_tuples_values[start_index:end_index+1]) # Extracts the start and end index
    globals()["times"+str(i)] = times # Names start and end index as times[i] and values[i]
    globals()["values"+str(i)] = values # Names start and end index as times[i] and values[i]
  except KeyError:
    continue

## Plotting all of our lines on the data

In [ ]:
nrows = math.ceil(num_row_half) # Setting no. of rows
ncols = 2 # Setting no. of columns

fig, axs = plt.subplots(nrows, ncols, figsize=(10, nrows*5)) # Creating the figure and subplots

axs = axs.flatten() # Flattening the subplot array to make it easier to iterate over - unsure if this is required

# Looping through the variables to plot each one
for i, ax in enumerate(axs): # Looping through 50 possible columns
    standardised_calcium_r = "standardised_calcium_r" + str(i+1) # Sets the variable to the one defined in the current iteration [i]
    values = "values" + str(i+1) # Sets the variable to the one defined in the current iteration [i]
    times = "times" + str(i+1) # Sets the variable to the one defined in the current iteration [i]
    if standardised_calcium_r in locals() and values in locals() and times in locals(): # Verifying that the variables are defined first
        ax.plot(time, locals()[standardised_calcium_r], 'b.') # Plotting time on the x axis
        ax.plot(locals()[times], locals()[values], 'r-') # Plotting our y axis and a red line denoting our max line
        ax.set_title("R{} calcium over time + fitted line (red)".format(i+1)) # Setting the title
        ax.set_ylabel("Calcium level".format(i+1)) # Setting the y axis label
        ax.set_xlabel("Time (sec)") # Setting the x axis label

## Making any reqired changes

### Refining tolerances

Once you've visualised the lines on the plots it's possible to edit the tolerance for each plot to try and improve the fit - this should only be required for a limited number of the plots as the default value works quite well! Increasing the tolerance should allow the line to get larger and work it's way through messier data, if the line is overstretched across the lag region or where the line begins to plateau it may be suitable o decrease the tolerance to improve the fit of the line.

Change the tolerance levels of specific standardised_calcium_r[i] by using the following format below:

(Do this as many times as required.)

In [ ]:
###As an example - above the line fitted to standardised_calcium_r13 seems to include the lag period which is visibly
#          skewing the gradient.###


tolerance12 = 0.4 ### Increaseing the tolerance for 12 will fit a longer line and include more of the region we're after.

Now go back up and run the cell titled '# Running the function...' and the next cell to re-plot the data and check your fit.

Either make further adjustments or continue on to calculate the overall average gradient for this data sheet.
   
###The standardised_calcium_r12 line is looking better! Time to continue... (no need to use our last resort!)###

### Last resort

As a last resort, if a line cannot be fitted to the graph it is possible to remove that cell by:

Removing the # below and replace [i] by the standardised_calcium_r[i] of interest.
        
This may reduce the acuracy of the overall average gradient for this cell type calculated below.

In [ ]:
#del average_gradient[i]

# *No need to do this as all our lines are plotted quite nicely!*

## Calculating the overall average gradient

In [ ]:
average_decline_gradients = {} # Creating a dictionary for our average_decline_gradients for each standardised_calcium_r[i]

# Looping through the variables and adding their average decline gradients to the dictionary
for i in range(1, 51): # Looping through 50 possible columns
    gradient = "average_gradient" + str(i) # Sets the variable to the one defined in the current iteration [i]
    if gradient not in locals():
        continue
    average_decline_gradients[i] = locals()[gradient]


# Defining a function to take an average of all the numbers in a dictionary.   
def average_gradient(dictionary): # Defining the function with no arguments
    total_sum = 0 # Setting our variables to 0
    num_elements = 0 # Setting our variables to 0
    for key, value in dictionary.items(): # Loops through each value
        total_sum += value # Adds the value to the total value
        num_elements += 1 # Adds 1 to the number of elements
    return total_sum / num_elements, num_elements # Returns the total sum devided by the number of elements, and defines
            # the number of elements

# Using this function to calculate the overall average gradient from the average_decline_gradients dictionary.
overall_average_gradient, num_elements = average_gradient(average_decline_gradients)

print("Invidual cell gradients are:\n")
for value in average_decline_gradients.values():
    print(value)

print("\n\nThe overall average gradient of calcium recovery of these", num_elements, "cells is: \n", overall_average_gradient)

# ### Success!
### ### The avereage gradient can now be compared with other disease-states and analysed further! ###
###Thank you for looking through my code!###